# Arase orbit - Direct read from local: 2024/2/27

In [ ]:
import pyspedas
import pytplot
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# from pytplot import tplot, cdf_to_tplot, data_quants, split_vec
import os
import urllib.request
import xarray as xr
import requests
from bs4 import BeautifulSoup

# Plot date & mode

In [ ]:
# Mode
mode_geometry = 0          # 0: SM (L, MLat, MLT)    1: GSE

# Date
year  = '2024'
month = '08'
day   = '20'

# Data folder
download_folder = f'/Users/user/OneDrive-Univ/data/data-Arase/erg_data/satellite/erg/orb/lpre/{year}/'

## Direct download

In [ ]:
# Direct read case
"""
# Original data directory (Long Predict Orbit)
url = f'https://ergsc.isee.nagoya-u.ac.jp/data/ergsc/satellite/erg/orb/lpre/{year}/'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Orbit folder
if not os.path.exists(download_folder):
    os.makedirs(download_folder)

# Download
for link in soup.find_all('a'):
    href = link.get('href')
    if href and year + month + day in href:
        download_url = url + href
        download_path = os.path.join(download_folder, href)

        print(f'Downloading {download_url} to {download_path}')
        with requests.get(download_url, stream=True) as r:
            with open(download_path, 'wb') as f:
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunk)
"""

## Direct read

In [ ]:
# Direct read
local_repository_data_path = download_folder+'/erg_orb_lpre_l2_{}{}{}_v02.cdf'.format(year, month, day)
pytplot.cdf_to_tplot(local_repository_data_path)

# Plot

In [ ]:
# GSE 
pytplot.split_vec('pos_gse')
x_gse = pytplot.data_quants['pos_gse_x']
y_gse = pytplot.data_quants['pos_gse_y']
z_gse = pytplot.data_quants['pos_gse_y']

###　一時間ごとに色分けしてプロットする　必要ない場合はx_gseとy_gseのみをプロットしてください　###
# xarrayのインデックスをDatetime64形式に変換
x_gse['time'] = pd.to_datetime(x_gse['time'].values)

In [ ]:
# Event time def
start_time = pd.Timestamp('2024-08-20 20:00:00')
mid1_time = pd.Timestamp('2024-08-20 21:00:00')
mid2_time = pd.Timestamp('2024-08-20 22:00:00')
mid3_time = pd.Timestamp('2024-08-20 23:00:00')
end_time = pd.Timestamp('2024-08-20 23:59:00')

# Event data pick up
x_20_21 = x_gse.sel(time=slice(start_time, mid1_time)); x_21_22 = x_gse.sel(time=slice(mid1_time, mid2_time))
x_22_23 = x_gse.sel(time=slice(mid2_time, mid3_time));  x_23_24 = x_gse.sel(time=slice(mid3_time, end_time))
y_20_21 = y_gse.sel(time=slice(start_time, mid1_time)); y_21_22 = y_gse.sel(time=slice(mid1_time, mid2_time))
y_22_23 = y_gse.sel(time=slice(mid2_time, mid3_time));  y_23_24 = y_gse.sel(time=slice(mid3_time, end_time)) 

In [ ]:
plt.figure(figsize=(5, 5))

# Axis
plt.xlim([6, -6]);  plt.ylim([6, -6])
plt.xlabel('Xgse [Re]', fontsize=16); plt.ylabel('Ygse [Re]', fontsize=16)

# plot
plt.plot(x_gse, y_gse, linewidth=1, color='black', linestyle='--', alpha=0.5, label='Arase')
plt.plot(x_20_21, y_20_21, linewidth=2, color=plt.cm.viridis(1/4), label='UT20-21', linestyle='-')
plt.plot(x_21_22, y_21_22, linewidth=2, color=plt.cm.viridis(2/4), label='UT21-22', linestyle='-')
plt.plot(x_22_23, y_22_23, linewidth=2, color=plt.cm.viridis(3/4), label='UT22-23', linestyle='-')
plt.plot(x_23_24, y_23_24, linewidth=2, color=plt.cm.viridis(4/4), label='UT23-24', linestyle='-') 
plt.legend(loc='lower right', fontsize=10)

# Earth
circle = plt.Circle((0, 0), 1, fill=False, color='black', linewidth=2)
plt.gca().add_patch(circle)

# Title
title_label = 'Arase ({}/{}/{})'.format(year, month, day)
plt.title(title_label)

# Grid
plt.grid(True)

plt.show()